In [1]:

# import conventions
include("conventions.jl")
  using .conventions: big_endian, qubit_begin

  # import quantum gates
include("quantum_gates.jl")
using ..quantum_gates: Qgate, Rz_gate1

include("lib_tensor/QTensor.jl")
using ..QTensor: Qgate_T2D


include("lib_useful/custom_functions.jl")
using ..custom_functions: MK_sortrows

include("quantum_circuit.jl")
using ..quantum_circuit: qc_init, init_register, show_statevector, op
#using ..quantum_circuit: qc_init, init_register, print_initstate

Load quantum gates constructor


Load Tensor module: QTensor.jl
Load quantum gates constructor
Load quantum_circuit constructor


Load quantum_circuit constructor
Load quantum gates constructor
Load quantum_circuit constructor


In [2]:
function p_phase(lambda)
    P = [1 0; 0 exp(im*lambda)]
    return P
end

p_phase (generic function with 1 method)

In [3]:
p_phase(2)

2×2 Matrix{ComplexF64}:
 1.0+0.0im        0.0+0.0im
 0.0+0.0im  -0.416147+0.909297im

In [4]:
Qgate.P(2)

2×2 Matrix{ComplexF64}:
 1.0+0.0im        0.0-0.0im
 0.0+0.0im  -0.416147+0.909297im

In [13]:
qc = qc_init(1)

Main.quantum_circuit.qc_initstruct(1, "big-endian", 2, 2, [1.0, 0.0], [0, 1], true, false)

In [11]:

# Apply a quantum gate to the quantum register
function op_test(qc, Qgate)
    # Apply a quantum gate to the quantum register
    # qc::quantum register
    # gate::Qgate: quantum gate
    # return: quantum register with the quantum gate applied
    Nqubits = qc.n_qubits
    Nstates = qc.n_dim
    state_vector = qc.state_vector
    # First check if the dimensions of the quantum gate are the same 
    Qgate_dim = size(Qgate)
    if Qgate_dim[1] != Qgate_dim[2]
        error("The quantum gate is not square")
    end # end if
    # check if the quantum gate is unitary
    #if ishermitian(Qgate) == false
    #    error("The quantum gate is not unitary")
    #end # end if
    # check the unitary condition
    UU = Qgate'*Qgate
    II = Matrix(I, Qgate_dim[1], Qgate_dim[2])
    if isapprox(UU, II,rtol=err_tol) == false
        error("The gate is not unitary")
    end
    # check if the dimensions of the quantum gate 
    # ... and the quantum register do match 
    if Qgate_dim[1] != Nstates
        error("The quantum gate and the quantum register do not match")
    end # end if
    # Apply the quantum gate to the quantum register
    state_vector = Qgate * state_vector
    qc.state_vector = state_vector
    return qc
end # end apply_gate!


op_test (generic function with 1 method)

In [14]:
H = Qgate.H
op_test(qc, Qgate)

MethodError: MethodError: no method matching size(::Main.quantum_gates.qgate)

Closest candidates are:
  size(!Matched::Union{LinearAlgebra.Adjoint{T, var"#s971"}, LinearAlgebra.Transpose{T, var"#s971"}} where {T, var"#s971"<:(AbstractVector)})
   @ LinearAlgebra /Applications/Julia-1.9.app/Contents/Resources/julia/share/julia/stdlib/v1.9/LinearAlgebra/src/adjtrans.jl:296
  size(!Matched::Union{LinearAlgebra.Adjoint{T, var"#s971"}, LinearAlgebra.Transpose{T, var"#s971"}} where {T, var"#s971"<:(AbstractMatrix)})
   @ LinearAlgebra /Applications/Julia-1.9.app/Contents/Resources/julia/share/julia/stdlib/v1.9/LinearAlgebra/src/adjtrans.jl:297
  size(!Matched::Union{LinearAlgebra.QR, LinearAlgebra.QRCompactWY, LinearAlgebra.QRPivoted})
   @ LinearAlgebra /Applications/Julia-1.9.app/Contents/Resources/julia/share/julia/stdlib/v1.9/LinearAlgebra/src/qr.jl:581
  ...


In [ ]:
function Unitary(theta)
    qc = qc_init(1, false)
    p = Qgate.P(pi*2*theta)
    op(qc,p)
    return qc
end

In [ ]:
theta = 1/2 + 1/4 + 1/8
unitary = Unitary(theta)

In [ ]:
qc = qc_init(2)

In [ ]:
show_statevector(unitary)

In [ ]:
CtrlU_be = Qgate.ctrl_gate

In [ ]:
function f_test(H_op; nqubits=1, convention=big_endian)
    qc = qc_init(nqubits, convention)
    op(qc, H_op)
    show_statevector(qc)
end

In [ ]:
H_op = Rz_gate1(π/2)

In [ ]:
f_test(H_op; nqubits=1, convention=big_endian)

In [ ]:
f_test(H_op; convention=big_endian, nqubits=1)

In [ ]:
# define a function that takes string as input for the name of the gates
function f_test1(H::String; nqubits=2, convention=big_endian)
    if H == "H"
        H_op = [1 1; 1 -1]/sqrt(2)
    elseif H == "X"
        H_op = [0 1; 1 0]
    elseif H == "Y"
        H_op = [0 -im; im 0]
    end 
    qc = qc_init(nqubits, convention)
    op(qc, H_op)
    show_statevector(qc)
end


In [ ]:
f_test1("H"; nqubits=1, convention=big_endian)

In [ ]:
qc = qc_init(2, big_endian)


In [ ]:
## create a table 
# data structure 
# converting our simulator to other simulators: TKET, QASM, Qiskit, ...etc. 
# array 
[action gate/operator control1_qubit,   control2_qubit, target_qubit theta     phi     lambda;
    1     "H"           nothing               nothing         0    nothing  nothing  nothing;
    2     "CX"        0                     nothing         1    nothing  nothing  nothing;
    3     "CX"        1                     nothing         2    nothing  nothing  nothing;
    4      "X"          nothing               nothing         1                             ;
    5      "Y"          nothing               nothing         2                             ;
    6      "Z"          nothing               nothing         3                             ;
    7      "S"          nothing               nothing         4                             ;
   8      "CU"         0                     nothing         2     pi/2      pi/3      pi/4;
    ]
   # n_qubits
    # barrier gate: indentity with the tag plotted on the circuit 1, when drawing the circuit 
    # it should appear in the plot. 
    # **************************
    # convention
    # op: taget qubit, control1 qubit, control2 qubit, ...



    function plot_circuit(qc; nqubits )
        # plot the circuit
        # plot the statevector
        nqubits = qc.n_qubits
        convention = qc.convention
        gate_table = qc.gate_table


    end

    # references 
    # https://typedtables.juliadata.org/stable/man/table/

In [15]:

H = Qgate.H
X = Qgate.X
Y = Qgate.Y
Z = Qgate.Z
S = Qgate.S

op(qc, Qgate.H(qtarget = 0))



MethodError: MethodError: objects of type Matrix{Float64} are not callable
Use square brackets [] for indexing an Array.

In [ ]:
import Pkg; Pkg.add("TypedTables")
using TypedTables


In [ ]:
`import Pkg; Pkg.add("TypedTables")`
using TypedTables

julia> t = Table(a = [1, 2, 3], b = [2.0, 4.0, 6.0])
Table with 2 columns and 3 rows:
     a  b
   ┌───────
 1 │ 1  2.0
 2 │ 2  4.0
 3 │ 3  6.0

julia> t[1]  # Get first row
(a = 1, b = 2.0)

julia> t.a  # Get column `a`
3-element Array{Int64,1}:
 1
 2
 3